In [ ]:
# pyriemann import
from pyriemann.classification import MDM, TSclassifier
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import butter, lfilter, freqz, iirnotch, filtfilt
import scipy.stats
import sklearn.decomposition
import pywt


In [ ]:
# Gumpy
# copy pasted the classes because package wouldnt import 
# is a Python 3 toolbox to develop Brain-Computer Interfaces (BCI).
# gumpy contains implementations of several functions that are commonly used during EEG and EMG decoding. For this purpose it heavily relies on other numerical and scientific libraries, 
# for instance numpy, scipy, or scikit-learn, to name just a few. In fact, gumpy mostly wraps existing functions in such a way that researchers
# working in the field can quickly perform data analysis and implement novel classifiers. Moreover, one of gumpy's design principles was to make it easily extendable.

from abc import ABC, abstractmethod
import numpy as np



class DatasetError(Exception):
    pass


class Dataset(ABC):
    """
    Abstract base class representing a dataset.
    All datasets should subclass from this baseclass and need to implement the
    `load` function. Initializing of the dataset and actually loading the data is
    separated as the latter may require significant time, depending on where the
    data is coming from. It also allows to implement different handlers for the
    remote end where the data originates, e.g. download from server, etc.
    When subclassing form Dataset it is helpful to set fields `data_type`,
    `data_name`, and `data_id`. For more information on this field, see for
    instance the implementation in :func:`gumpy.data.graz.GrazB.__init__`.
    """


    def __init__(self, **kwargs):
        """Initialize a dataset."""
        pass


    @abstractmethod
    def load(self, **kwargs):
        """Load the data and prepare it for usage.
        gumpy expects the EEG/EMG trial data to be in the following format:
            ===========================================> time
                |                                   |
            trial_start                         trial_end
                |<------------trial_len------------>|
                                |<---MotorImager--->|
        Consequentially the class members need to adhere the following structure
            .raw_data       (n_samples, n_channels)  return all channels
            .trials         (,n_trials)
            .labels         (,n_labels)
            .trial_len      scalar
            .sampling_freq  scalar
            .mi_interval    [mi_start, mi_end] within a trial in seconds
        Arrays, such as `.raw_data` have to be accessible using bracket
        notation `[]`. You can provide a custom implementation, however the
        easiest way is to use numpy ndarrays to store the data.
        For an example implementation, have a look at `gumpy.data.nst.NST`.
        """
        return self


    def print_stats(self):
        """Commodity function to print information about the dataset.
        This method uses the fields that need to be implemented when
        subclassing. For more information about the fields that need to be
        implemented see :func:`gumpy.data.dataset.Dataset.load` and
        :func:`gumpy.data.dataset.Dataset.__init__`.
        """

        print("Data identification: {name}-{id}".format(name=self.data_name, id=self.data_id))
        print("{type}-data shape: {shape}".format(type=self.data_type, shape=self.raw_data.shape))
        print("Trials data shape: ", self.trials.shape)
        print("Labels shape: ", self.labels.shape)
        print("Total length of single trial: ", self.trial_total)
        print("Sampling frequency of {type} data: {freq}".format(type=self.data_type, freq=self.sampling_freq))
        print("Interval for motor imagery in trial: ", self.mi_interval)
        print('Classes possible: ', np.unique(self.labels))


In [ ]:
# # Created on Mon Nov 14 17:36:59 2016
# # @author: coelhorp
# # https://github.com/plcrodrigues/PhD-Code/blob/2ff5642775dc56869994109fea7e0db90f5f9bdb/utilities/dimensionality_reduction.py
# import scipy as sp

# from sklearn.decomposition import PCA
# from sklearn.base import BaseEstimator, TransformerMixin

# from pyriemann.utils.base     import powm, sqrtm, invsqrtm, logm
# from pyriemann.utils.distance import distance_riemann, distance_logeuclid
# from pyriemann.utils.mean     import mean_riemann, mean_euclid

# from random import randrange

# class RDR(BaseEstimator, TransformerMixin):    
#     '''Riemannian Dimension Reduction
    
#     Dimension reduction respecting the riemannian geometry of the SPD manifold.
    
#     Parameters
#     ----------
#     n_components: int (default: 6)
#         The number of components to reduce the dataset. 
#     method: string (default: nrme)
#         Which method should be used to reduce the dimension of the dataset.
#         Different approaches use different cost functions and algorithms for
#         solving the optimization problem. The options are:        
#             - rme-uns
#             - rme-uns-bm (set nmeans, npoints)
#             - covpca             
#     '''
    
#     def __init__(self, n_components=6, method='nrme', params={}):          
#         self.n_components = n_components
#         self.method = method
#         self.params = params
        
#     def fit(self, X, y=None):        
#         self._fit(X, y)
#         return self

#     def transform(self, X, y=None):        
#         Xnew = self._transform(X)
#         return Xnew
        
#     def _fit(self, X, y):   
             
#         methods = {
#                    'rme-uns'        : dim_reduction_nrmeuns,
#                    'rme-uns-bm' : dim_reduction_nrmeuns_random,               
#                    'covpca'          : dim_reduction_covpca,                   
#                   }    
                                   
#         self.projector_ = methods[self.method](X=X,
#                                                P=self.n_components,
#                                                labels=y,
#                                                params=self.params)                                         
    
#     def _transform(self, X):        
#         K = X.shape[0]
#         P = self.n_components
#         W = self.projector_    
#         Xnew = np.zeros((K, P, P))        
#         for k in range(K):            
#             Xnew[k, :, :] = np.dot(W.T, np.dot(X[k, :, :], W))                        
#         return Xnew 

In [ ]:
# gumpy 
# import wouldnt work, classes from github
# 
# filter 
class ButterBandpass:
    """Filter class for a Butterworth bandpass filter.
    """

    def __init__(self, lowcut, highcut, order=4, fs=256):
        """Initialize the Butterworth bandpass filter.
        Args:
            lowcut (float): low cut-off frequency
            highcut (float): high cut-off frequency
            order (int): order of the Butterworth bandpass filter
            fs (int): sampling frequency
        """
        self.lowcut = lowcut
        self.highcut = highcut
        self.order = order

        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        self.b, self.a = scipy.signal.butter(order, [low, high], btype='bandpass')


    def process(self, data, axis=0):
        """Apply the filter to data along a given axis.
        Args:
            data (array_like): data to filter
            axis (int): along which data to filter
        Returns:
            ndarray: Result of the same shape as data
        """
        return scipy.signal.filtfilt(self.b, self.a, data, axis)

def butter_bandpass(data, lo, hi, axis=0, **kwargs):
    """Apply a Butterworth bandpass filter to some data.
    The function either takes an ``array_like`` object (e.g. numpy's ndarray) or
    an instance of a gumpy.data.Dataset subclass as first argument.
    Args:
        data (array_like or Dataset instance): input data. If this is an
            instance of a Dataset subclass, the sampling frequency will be extracted
            automatically.
        lo (float): low cutoff frequency.
        hi (float): high cutoff frequency.
        axis (int): along which axis of data the filter should be applied. Default = 0.
        **kwargs: Additional keyword arguments that will be passed to ``gumpy.signal.ButterBandstop``.
    Returns:
        array_like: data filtered long the specified axis.
    """
    if isinstance(data, Dataset):
        flt = ButterBandpass(lo, hi, fs=data.sampling_freq, **kwargs)
        filtered = [flt.process(data.raw_data[:, i], axis) for i in range(data.raw_data.shape[1])]
        reshaped = [f.reshape(-1, 1) for f in filtered]
        return np.hstack(reshaped)
    else:
        flt = ButterBandpass(lo, hi, **kwargs)
        return flt.process(data, axis)    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
copied_path = 'drive/My Drive/DEAP_data/' #remove ‘content/’ from path then use 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
pip install pyriemann

     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
pip install gumpy

     |████████████████████████████████| 3.0MB 3.4MB/s 
     |████████████████████████████████| 10.8MB 24.1MB/s 
     |████████████████████████████████| 256kB 41.2MB/s 
     |████████████████████████████████| 2.3MB 23.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
pip install scipy

In [ ]:
import scipy.io

In [ ]:
import numpy as np
import gumpy
from pyriemann.estimation import Covariances, Shrinkage
from pyriemann.classification import TSclassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


class Frequencies(object):

    def __init__(self, mean='logeuclid', distance='riemann'):
        self.mean = mean
        self.distance = distance
        self.clf = TSclassifier(metric='riemann', clf=LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'))
        self.low = 15
        self.high = 36
        # 10 fold cv
        self.cv = KFold(10, shuffle=True, random_state=42)

    def getAlfa_Beta_Gamma(self, X, s, y_valence, y_arousal):
        print("\n-3-Band-run-")
        # 3 band filtering the data
        # frequencies between those values are filtered out 
        filtered_data = butter_bandpass(X, lo=self.low, hi=self.high)
        # compute covariance matrices
        covariance_matrix_data = Covariances(estimator='lwf').transform(filtered_data)
        covariance_matrix_data = Shrinkage().transform(covariance_matrix_data)
        valence_scores = cross_val_score(self.clf, covariance_matrix_data, y_valence, cv=self.cv)
        arousal_scores = cross_val_score(self.clf, covariance_matrix_data, y_arousal, cv=self.cv)

        return np.mean(valence_scores * 100), np.std(valence_scores), np.mean(arousal_scores * 100), np.std(arousal_scores)


In [ ]:
path = copied_path
low_valence = np.array(())
high_valence = np.array(())

low_arousal = np.array(())
high_arousal = np.array(())
#3
valence_accuracy = np.array(())
std_valence= np.array(())

arousal_accuracy = np.array(())
std_arousal = np.array(())

subjects = np.array(())

for s in range(1, 33):
    if s < 10:
        b = '0' + str(s)
    else:
        b = str(s)
    subject = path+'s'+b+'.mat'

    mat = scipy.io.loadmat(subject)
    X = np.array(mat['data'])
    X = X[:, :32, :]

    #SHAPE (40, 4)
    y = np.array(mat['labels'])

    #SHAPE (40, )
    y_valence = y[:, 0]
    y_valence = (y_valence > 5).astype(np.int_)

    y_arousal = y[:, 1]
    y_arousal = (y_arousal > 5).astype(np.int_)
    low_valence = np.append(low_valence, np.count_nonzero(y_valence == 0))
    high_valence = np.append(high_valence, np.count_nonzero(y_valence == 1))

    low_arousal = np.append(low_valence, np.count_nonzero(y_arousal == 0))
    high_arousal = np.append(high_valence, np.count_nonzero(y_arousal == 1))

    print("part_id:", s)
    print("Elements_low_val: ", np.count_nonzero(y_valence == 0))
    print("Elements_high_val: ", np.count_nonzero(y_valence == 1))
    print("Elements_low_arouse: ", np.count_nonzero(y_arousal == 0))
    print("Elements_high_arouse: ", np.count_nonzero(y_arousal == 1))
    
    

    valence_Score_Acc_R3, valence_STD_R3, arousal_Score_Acc_R3, arousal_STD_R3 = Frequencies().getAlfa_Beta_Gamma(X, s, y_valence, y_arousal)

    valence_accuracy = np.append(valence_accuracy, valence_Score_Acc_R3)
    std_valence = np.append(std_valence, valence_STD_R3)

    arousal_accuracy = np.append(arousal_accuracy, arousal_Score_Acc_R3)
    std_arousal = np.append(std_arousal, arousal_STD_R3)

    print("------------------------------------------------------------------------")
    print("Perfomance Mean Valence (3): ", np.mean(valence_accuracy), "STD: ", np.mean(std_valence))
    print("Perfomance Mean Arousal (3): ", np.mean(arousal_accuracy), "STD: ", np.mean(std_arousal))
    subjects = np.append(subjects, s)

output_low_valence= low_valence.sum()
output_high_valence = high_valence.sum()

output_low_arousal = low_arousal.sum()
output_high_arousal= high_arousal.sum()


print("\nTotal Class 0 (LOW VALENCE): ", output_low_valence)
print("Total Class 1 (HIGH VALENCE): ", output_high_valence )

print("\nTotal Class 0 (LOW AROUSAL): ", output_low_arousal)
print("Total Class 1 (HIGH AROUSAL): ", output_high_arousal)







part_id: 1
Elements_low_val:  21
Elements_high_val:  19
Elements_low_arouse:  16
Elements_high_arouse:  24

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  85.0 STD:  0.12247448713915891
Perfomance Mean Arousal (3):  77.5 STD:  0.13462912017836262
part_id: 2
Elements_low_val:  18
Elements_high_val:  22
Elements_low_arouse:  16
Elements_high_arouse:  24

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  86.25 STD:  0.16201546542331133
Perfomance Mean Arousal (3):  66.25 STD:  0.1423145600891813
part_id: 3
Elements_low_val:  18
Elements_high_val:  22
Elements_low_arouse:  32
Elements_high_arouse:  8

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  82.5 STD:  0.18254590953220054
Perfomance Mean Arousal (3):  70.83333333333333 STD:  0.14487637339278756
part_id: 4
Elements_low_val:  24
Elemen

In [ ]:
63.96

In [ ]:
from pyriemann.classification import FgMDM

In [ ]:
class FrequenciesMDM(object):

    def __init__(self, mean='logeuclid', distance='riemann'):
        self.mean = mean
        self.distance = distance
        self.clf = FgMDM(metric='riemann', n_jobs = -1)
        self.low = 15
        self.high = 36
        # 10 fold cv
        self.cv = KFold(10, shuffle=True, random_state=42)

    def getAlfa_Beta_Gamma(self, X, s, y_valence, y_arousal):
        print("\n-3-Band-run-")
        # 3 band filtering the data
        # frequencies between those values are filtered out 
        filtered_data = butter_bandpass(X, lo=self.low, hi=self.high)
        # compute covariance matrices
        covariance_matrix_data = Covariances(estimator='lwf').transform(filtered_data)
        covariance_matrix_data = Shrinkage().transform(covariance_matrix_data)
        valence_scores = cross_val_score(self.clf, covariance_matrix_data, y_valence, cv=self.cv)
        arousal_scores = cross_val_score(self.clf, covariance_matrix_data, y_arousal, cv=self.cv)

        return np.mean(valence_scores * 100), np.std(valence_scores), np.mean(arousal_scores * 100), np.std(arousal_scores)


In [ ]:
path = copied_path
low_valence = np.array(())
high_valence = np.array(())

low_arousal = np.array(())
high_arousal = np.array(())
#3
valence_accuracy = np.array(())
std_valence= np.array(())

arousal_accuracy = np.array(())
std_arousal = np.array(())

subjects = np.array(())

for s in range(1, 33):
    if s < 10:
        b = '0' + str(s)
    else:
        b = str(s)
    subject = path+'s'+b+'.mat'

    mat = scipy.io.loadmat(subject)
    X = np.array(mat['data'])
    X = X[:, :32, :]

    #SHAPE (40, 4)
    y = np.array(mat['labels'])

    #SHAPE (40, )
    y_valence = y[:, 0]
    y_valence = (y_valence > 5).astype(np.int_)

    y_arousal = y[:, 1]
    y_arousal = (y_arousal > 5).astype(np.int_)
    low_valence = np.append(low_valence, np.count_nonzero(y_valence == 0))
    high_valence = np.append(high_valence, np.count_nonzero(y_valence == 1))

    low_arousal = np.append(low_valence, np.count_nonzero(y_arousal == 0))
    high_arousal = np.append(high_valence, np.count_nonzero(y_arousal == 1))

    print("part_id:", s)
    print("Elements_low_val: ", np.count_nonzero(y_valence == 0))
    print("Elements_high_val: ", np.count_nonzero(y_valence == 1))
    print("Elements_low_arouse: ", np.count_nonzero(y_arousal == 0))
    print("Elements_high_arouse: ", np.count_nonzero(y_arousal == 1))
    
    

    valence_Score_Acc_R3, valence_STD_R3, arousal_Score_Acc_R3, arousal_STD_R3 = FrequenciesMDM().getAlfa_Beta_Gamma(X, s, y_valence, y_arousal)

    valence_accuracy = np.append(valence_accuracy, valence_Score_Acc_R3)
    std_valence = np.append(std_valence, valence_STD_R3)

    arousal_accuracy = np.append(arousal_accuracy, arousal_Score_Acc_R3)
    std_arousal = np.append(std_arousal, arousal_STD_R3)

    print("------------------------------------------------------------------------")
    print("Perfomance Mean Valence (3): ", np.mean(valence_accuracy), "STD: ", np.mean(std_valence))
    print("Perfomance Mean Arousal (3): ", np.mean(arousal_accuracy), "STD: ", np.mean(std_arousal))
    subjects = np.append(subjects, s)

output_low_valence= low_valence.sum()
output_high_valence = high_valence.sum()

output_low_arousal = low_arousal.sum()
output_high_arousal= high_arousal.sum()


print("\nTotal Class 0 (LOW VALENCE): ", output_low_valence)
print("Total Class 1 (HIGH VALENCE): ", output_high_valence )

print("\nTotal Class 0 (LOW AROUSAL): ", output_low_arousal)
print("Total Class 1 (HIGH AROUSAL): ", output_high_arousal)







part_id: 1
Elements_low_val:  21
Elements_high_val:  19
Elements_low_arouse:  16
Elements_high_arouse:  24

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  85.0 STD:  0.12247448713915891
Perfomance Mean Arousal (3):  75.0 STD:  0.15811388300841897
part_id: 2
Elements_low_val:  18
Elements_high_val:  22
Elements_low_arouse:  16
Elements_high_arouse:  24

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  86.25 STD:  0.16201546542331133
Perfomance Mean Arousal (3):  66.25 STD:  0.17668506245304266
part_id: 3
Elements_low_val:  18
Elements_high_val:  22
Elements_low_arouse:  32
Elements_high_arouse:  8

-3-Band-run-
------------------------------------------------------------------------
Perfomance Mean Valence (3):  82.5 STD:  0.18254590953220054
Perfomance Mean Arousal (3):  70.0 STD:  0.1626664150281493
part_id: 4
Elements_low_val:  24
Elements_high_val: 

In [ ]:
slight improvement in performance 
the other one is wya 